***GENERATED CODE FOR creditcardfrauddetectionclassification PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import round
from pyspark.ml.feature import Binarizer
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def BinarizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    outcol = feature + "_binarizer"
    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature)\
        .withColumnRenamed("feature_cast", feature)

    dfReturn = dfReturn.fillna({feature: 0.0})
    binarizer = Binarizer(threshold=float(
        transformationData['threshold']), inputCol=feature, outputCol=outcol)
    binarizedDataFrame = binarizer.transform(dfReturn)

    # binarizedDataFrame=binarizedDataFrame.drop(feature).withColumnRenamed(outcol,feature)

    dfReturn = binarizedDataFrame
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Time', 'threshold': 137555.49, 'transformation_label': 'Binarizer'}], 'feature': 'Time', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
                                              'count': '500', 'mean': '177.04', 'stddev': '108.03', 'min': '0.0', 'max': '368.0', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'Time'}, {'feature_label': 'Time', 'threshold': 137555.49, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('Time')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run creditcardfrauddetectionclassificationHooks.ipynb
try:
	#sourcePreExecutionHook()

	creditcard = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/Data/Finance_Models/creditcard.csv', 'filename': 'creditcard.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://10.0.5.54', 'port': '50070', 'dirPath': '/FileStore/platform', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(creditcard)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run creditcardfrauddetectionclassificationHooks.ipynb
try:
	#transformationPreExecutionHook()

	creditcardfrauddetectionclassificationautofe = TransformationMain.run(creditcard,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Time", "threshold": 137555.49, "transformation_label": "Binarizer"}], "feature": "Time", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "177.04", "stddev": "108.03", "min": "0.0", "max": "368.0", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "Time"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V1", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.38", "stddev": "1.63", "min": "-11.06394773", "max": "1.510050223", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V1"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V2", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.24", "stddev": "1.57", "min": "-13.53698545", "max": "6.083501467", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V3", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.76", "stddev": "1.39", "min": "-9.514274774", "max": "3.675173879", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V3"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V4", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.11", "stddev": "1.7", "min": "-4.082060878", "max": "11.19089466", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V4"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V5", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.36", "stddev": "1.22", "min": "-14.5447087", "max": "3.539277073", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V5"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V6", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.25", "stddev": "1.3", "min": "-3.498447175", "max": "5.122102581", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V6"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V7", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.53", "stddev": "1.92", "min": "-17.29065667", "max": "2.744854347", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V7"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V8", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.02", "stddev": "1.42", "min": "-17.91341381", "max": "6.85600274", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V8"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V9", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.26", "stddev": "0.94", "min": "-3.260671529", "max": "2.920155991", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V9"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V10", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.15", "stddev": "0.99", "min": "-2.732728958", "max": "8.359879351", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V10"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V11", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.24", "stddev": "0.86", "min": "-2.413889834", "max": "1.846585429", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V11"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V12", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.26", "stddev": "0.64", "min": "-2.36104985", "max": "2.152055116", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V12"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V13", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.11", "stddev": "0.94", "min": "-3.420438754", "max": "1.837795211", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V13"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V14", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.07", "stddev": "0.98", "min": "-6.6190902", "max": "3.886382393", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V14"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V15", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.22", "stddev": "0.92", "min": "-3.618059825", "max": "2.345864949", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V15"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V16", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.0", "stddev": "0.86", "min": "-3.176844479", "max": "3.048982231", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V16"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V17", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.03", "stddev": "0.73", "min": "-2.090586302", "max": "3.13370478", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V17"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V18", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.01", "stddev": "0.76", "min": "-3.104560956", "max": "2.492989466", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V18"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V19", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.09", "stddev": "0.89", "min": "-3.409760266", "max": "3.8693304", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V19"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V20", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.04", "stddev": "0.44", "min": "-4.516220794", "max": "2.521740678", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V20"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V21", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.09", "stddev": "0.61", "min": "-4.106145032", "max": "5.500362423", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V21"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V22", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.09", "stddev": "0.65", "min": "-8.593641565", "max": "1.985148506", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V22"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V23", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.01", "stddev": "0.73", "min": "-14.00087907", "max": "2.268002816", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V23"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V24", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.02", "stddev": "0.64", "min": "-2.221543136", "max": "3.951679291", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V24"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V25", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.13", "stddev": "0.43", "min": "-1.389079168", "max": "1.136720149", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V25"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V26", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.06", "stddev": "0.52", "min": "-2.534329721", "max": "1.55317104", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V26"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V27", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.01", "stddev": "0.35", "min": "-3.244847857", "max": "2.081686935", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V27"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V28", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.03", "stddev": "0.34", "min": "-2.093846147", "max": "1.146647647", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V28"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Amount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "69.72", "stddev": "217.9", "min": "0.0", "max": "3828.04", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Amount"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Class", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.04", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Class"}]}))

	#transformationPostExecutionHook(creditcardfrauddetectionclassificationautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run creditcardfrauddetectionclassificationHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(creditcardfrauddetectionclassificationautofe, ["V1", "V2", "V3", "V4", "V5", "V6", "V7", "V8", "V9", "V10", "V11", "V12", "V13", "V14", "V15", "V16", "V17", "V18", "V19", "V20", "V21", "V22", "V23", "V24", "V25", "V26", "V27", "V28", "Amount", "Time_binarizer"], "Class")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

